In [6]:
import numpy as np
import pandas as pd
!pip install yfinance
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import norm

In [7]:
# Step 1: Fetch Historical Data
print("Fetching stock data...")
stocks = ["AARTIPHARM.NS", "BLUESTARCO.NS", "FEDERALBNK.NS", "ICIL.NS", "IEX.NS", 
    "ITC.NS", "MARKSANS.NS", "RAYMOND.NS", "NITINSPIN.NS", "RALLIS.NS",
    "APOLLO.NS", "WINDLAS.NS", "NATCOPHARM.NS"]
data = yf.download(stocks, start='2024-01-14', end='2025-01-15')['Adj Close']

Fetching stock data...


[*********************100%***********************]  13 of 13 completed


In [8]:
data.head(2)

Ticker,AARTIPHARM.NS,APOLLO.NS,BLUESTARCO.NS,FEDERALBNK.NS,ICIL.NS,IEX.NS,ITC.NS,MARKSANS.NS,NATCOPHARM.NS,NITINSPIN.NS,RALLIS.NS,RAYMOND.NS,WINDLAS.NS
Date,,,,,,,,,,,,,
2024-01-15,531.281982,123.291824,1057.964233,152.047272,285.219971,160.951385,452.800629,160.421616,844.892944,329.169678,253.260696,1827.447510,480.452301
2024-01-16,516.811462,122.892014,1046.860229,148.816467,279.155701,161.393967,457.351807,159.872742,828.313660,328.175659,258.063507,1829.887939,460.924103


In [9]:
# Calculate daily returns
returns = data.pct_change().dropna()
returns.head(2)

Ticker,AARTIPHARM.NS,APOLLO.NS,BLUESTARCO.NS,FEDERALBNK.NS,ICIL.NS,IEX.NS,ITC.NS,MARKSANS.NS,NATCOPHARM.NS,NITINSPIN.NS,RALLIS.NS,RAYMOND.NS,WINDLAS.NS
Date,,,,,,,,,,,,,
2024-01-16,-0.027237,-0.003243,-0.010496,-0.021249,-0.021262,0.002750,0.010051,-0.003421,-0.019623,-0.003020,0.018964,0.001335,-0.040645
2024-01-17,-0.012508,-0.009760,0.010036,-0.025384,-0.015135,-0.102681,-0.012068,-0.039638,-0.016289,0.006664,-0.014774,-0.012547,0.032881


In [68]:
# Portfolio weights (as provided in your data)
weights = np.array([4.25, 4.87, 7.44, 5.58, 8.95, 11.02, 6.64, 9.84, 6.69, 
                    10.10, 7.12, 10.23, 7.53]) / 100  # Converting percentage to decimal

In [70]:
# Portfolio daily returns
portfolio_returns = returns.dot(weights).dropna()
portfolio_returns

Date
2024-01-16   -0.007743
2024-01-17   -0.019325
2024-01-18   -0.004203
2024-01-19    0.008072
2024-01-23   -0.026028
                ...   
2025-01-08   -0.013331
2025-01-09   -0.015484
2025-01-10   -0.030229
2025-01-13   -0.032191
2025-01-14    0.024414
Length: 245, dtype: float64

In [72]:
portfolio_returns = pd.DataFrame(portfolio_returns)
portfolio_returns.head()

,0
Date,
2024-01-16,-0.007743
2024-01-17,-0.019325
2024-01-18,-0.004203
2024-01-19,0.008072
2024-01-23,-0.026028


In [98]:
# Step 2: Define VaR Calculation Methods
confidence_level = 0.95
investment_value = 107595  # Provided total investment
mean = portfolio_returns.mean()
std_dev = portfolio_returns.std()
z_score = norm.ppf(confidence_level) 

In [100]:
print(mean)
print(std_dev)
print(z_score)

0    0.001267
dtype: float64
0    0.015158
dtype: float64
1.6448536269514722


In [102]:
# Historical Simulation Method
ascending_returns = np.sort(portfolio_returns)
def var_historical(portfolio_returns, confidence):
    return np.percentile(portfolio_returns, (1 - confidence) * 100)
var_hist = var_historical(portfolio_returns, confidence_level)*investment_value
var_hist

-2866.0860403864062

In [104]:
# Variance-Covariance Method
var_vc = (mean - (std_dev*z_score)) * investment_value
var_vc

0   -2546.279959
dtype: float64

In [106]:
# Monte Carlo Simulation Method

#generate uncorrelate random variables
num_securities = 13
num_simulations = 1000
random_variables = np.random.normal(0,1,(num_simulations, num_securities))

#Convert an uncorrelated random variable to correlated random variable 
cov_matrix = returns.cov()
cholesky_matrix = np.linalg.cholesky(cov_matrix)
correlated_random_variables = np.dot(random_variables,cholesky_matrix.T)

#Required Parameters
initial_price = [531.28, 123.29, 1057.96, 152.04, 285.21, 160.95, 452.80, 160.42, 844.89, 329.16, 253.26, 1827.44, 480.45]
mu = [0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07, 0.07]
sigma = np.std(correlated_random_variables, axis=0)
for t in range(1, 1001):  # Loop over 1000 steps
    time_fraction = t / 1000

#Price simulation
price_simulation = initial_price * np.exp((mu - (sigma**2)/2) * time_fraction + sigma * correlated_random_variables)
price_simulation = pd.DataFrame(price_simulation)

#Simulated returns & portfolio returns
simulated_returns = price_simulation.pct_change().dropna()
sim_port_return = np.dot(simulated_returns, weights)

sorted_returns = np.sort(sim_port_return)
def var_monte_carlo(sorted_returns, confidence):
    return np.percentile(sorted_returns, (1 - confidence) * 100)

var_mc = var_monte_carlo(sorted_returns, confidence_level)*investment_value
var_mc

-117.586789683609

In [108]:
# Step 4: Stress Testing
print("Performing stress test...")
stress_factor = 2  # Assume a market scenario with 2x volatility
stress_portfolio_returns = portfolio_returns * stress_factor

Performing stress test...


In [110]:
print(var_hist)
print(var_vc)
print(var_mc)

-2866.0860403864062
0   -2546.279959
dtype: float64
-117.586789683609


In [112]:
portfolio_pnl = portfolio_returns * investment_value
portfolio_pnl.head(2)

,0
Date,
2024-01-16,-833.081088
2024-01-17,-2079.240724


In [116]:
breach = (var_hist > portfolio_pnl).sum()
breach

0    13
dtype: int64

In [118]:
mc_pnl = portfolio_returns * investment_value
mc_pnl.head(2)

,0
Date,
2024-01-16,-833.081088
2024-01-17,-2079.240724


In [124]:
mc_breach = (var_hist > portfolio_pnl).sum()
mc_breach

0    13
dtype: int64